In [14]:
import pandas as pd
from fireagg.database.db import connect

with connect() as commands:
    trades_res = commands.query("""
    SELECT symbol, connector, price, amount, is_buy, timestamp, t.update_timestamp, t.fetch_timestamp
    FROM symbol_trades_stream t
    JOIN symbols s ON s.id = symbol_id
    """)


trades = pd.DataFrame(trades_res)

In [15]:
import plotly.express as px

display(px.histogram(trades, x="timestamp"))

trades.sort_values('timestamp', inplace=True)
# data.to_parquet('connector_latency_v2.parquet', allow_truncated_timestamps=True, coerce_timestamps="us")

# Very slow
# display(px.line(data, x=data.index, y='mid_price', color='connector'))

In [3]:
import datetime
import plotly.express as px

downsampled = (
    data.set_index("timestamp", drop=False)
    .groupby(["connector", "symbol"])
    .resample("5T")
    .last()
)

for symbol in downsampled.symbol.unique():
    symbol_data = downsampled[downsampled.symbol == symbol]
    fig = px.line(
        symbol_data,
        x="timestamp",
        y="mid_price",
        color="connector",
        title=symbol,
        line_shape='hv'
    )
    display(fig)

In [5]:
import numpy as np
from filterpy.kalman import KalmanFilter, ExtendedKalmanFilter

data_symbol = data[data.symbol == 'CRV/USDT']
prices_by_connector = pd.pivot_table(data=data_symbol, index='timestamp', columns='connector', values='price')
prices_by_connector = prices_by_connector.ffill().bfill()

predictions = []
f = KalmanFilter(dim_x=1, dim_z=len(prices_by_connector.columns))
f.x = prices_by_connector.iloc[0].mean()
for ts, row in prices_by_connector.iterrows():
    f.predict()
    f.update(row.to_numpy())
    predictions.append(f.x[0][0])


In [6]:
predictions

[0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500000001,
 0.5920602500